<a href="https://colab.research.google.com/github/romitbarua/MultiModalDeepFake/blob/main/TestingNotebooks/metadata_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
metadata_file = r'/content/drive/MyDrive/FakeAVCeleb/FakeAVCeleb_v1.2/meta_data.csv'

In [ ]:
import pandas as pd

In [ ]:
metadata_df = pd.read_csv(metadata_file)

In [ ]:
relative_path = '/content/drive/MyDrive/FakeAVCeleb/FakeAVCeleb_v1.2/'

metadata_df =  metadata_df.rename({'path': 'filename', 'Unnamed: 9': 'path'}, axis='columns')

real_df = metadata_df[metadata_df.method.isin(['real'])]
wav2lip_df = metadata_df[metadata_df.method.isin(['real', 'wav2lip'])]


In [ ]:
def generate_file_path(relative_path, paths, filenames):
  path_terms_list = [path.split('/') for path in paths]
  path_terms_list = [path_terms[1:] for path_terms in path_terms_list]

  filepaths_list = []

  for filename, path_terms in zip(filenames, path_terms_list):
    filepaths_list.append(relative_path + '/'.join(path_terms) + '/' + filename)

  return filepaths_list

In [ ]:
paths = wav2lip_df.path.to_list()
filenames = wav2lip_df.filename.to_list()
filepaths_list = generate_file_path(relative_path, paths, filenames)

In [ ]:
#wav2lip_df['filepath'] = filepaths_list
wav2lip_df = wav2lip_df.assign(filepath = filepaths_list)

In [ ]:
print(wav2lip_df.iloc[-1])

source                                                id07689
target1                                               id07686
target2                                                     -
method                                                wav2lip
category                                                    D
type                                      FakeVideo-FakeAudio
race                                            Asian (South)
gender                                                  women
filename                           00028_id07686_wavtolip.mp4
path        FakeAVCeleb/FakeVideo-FakeAudio/Asian (South)/...
filepath    /content/drive/MyDrive/FakeAVCeleb/FakeAVCeleb...
Name: 21565, dtype: object


In [ ]:
#wav2lip_df[['filename','path','filepath']]

In [ ]:
real_df.head()

,source,target1,target2,method,category,type,race,gender,filename,path
0,id00076,-,-,real,A,RealVideo-RealAudio,African,men,00109.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...
1,id00166,-,-,real,A,RealVideo-RealAudio,African,men,00010.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...
2,id00173,-,-,real,A,RealVideo-RealAudio,African,men,00118.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...
3,id00366,-,-,real,A,RealVideo-RealAudio,African,men,00118.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...
4,id00391,-,-,real,A,RealVideo-RealAudio,African,men,00052.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...


In [ ]:
wav2lip_df.head()

,source,target1,target2,method,category,type,race,gender,filename,path,filepath
0,id00076,-,-,real,A,RealVideo-RealAudio,African,men,00109.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...,/content/drive/MyDrive/FakeAVCeleb/FakeAVCeleb...
1,id00166,-,-,real,A,RealVideo-RealAudio,African,men,00010.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...,/content/drive/MyDrive/FakeAVCeleb/FakeAVCeleb...
2,id00173,-,-,real,A,RealVideo-RealAudio,African,men,00118.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...,/content/drive/MyDrive/FakeAVCeleb/FakeAVCeleb...
3,id00366,-,-,real,A,RealVideo-RealAudio,African,men,00118.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...,/content/drive/MyDrive/FakeAVCeleb/FakeAVCeleb...
4,id00391,-,-,real,A,RealVideo-RealAudio,African,men,00052.mp4,FakeAVCeleb/RealVideo-RealAudio/African/men/id...,/content/drive/MyDrive/FakeAVCeleb/FakeAVCeleb...


In [ ]:
#we only need to consider target 1
len(wav2lip_df[wav2lip_df['method']=='wav2lip'].target1.unique()), len(wav2lip_df[wav2lip_df['method']=='wav2lip'].target2.unique())

(500, 1)

In [ ]:
paths2 = real_df.path.to_list()
filenames2 = real_df.filename.to_list()
filepaths_list2 = generate_file_path(relative_path, paths2, filenames2)
real_df = real_df.assign(filepath = filepaths_list2)

In [ ]:
def generate_utternace_path(filepaths):

  paths2 = real_df.filepath.to_list()
  path_terms_list = [path.split('/') for path in paths2]
  utterance_files = [path_terms_list[i][-1].split('.')[0] + '_text.txt' for i in range(len(path_terms_list))]
  utterance_file_path = []

  for path_terms, utterance_file in zip(path_terms_list, utterance_files):
    utterance_file_path.append('/'.join(path_terms[:-1]) + '/' + utterance_file)

  return utterance_file_path

In [ ]:
filepaths = real_df.filepath.to_list()
utterance_list = generate_utternace_path(filepaths)
real_df = real_df.assign(utterance_file = utterance_list)

In [ ]:
#merged_df = pd.merge(left=wav2lip_df, right=real_df[['source', 'filename', 'method', 'utterance_file']], how='left',
                      #left_on=['source', 'filename', 'method'], right_on=['source', 'filename', 'method'])

In [ ]:
real_df.loc[real_df['source'] == 'id07686', 'utterance_file']

498    /content/drive/MyDrive/FakeAVCeleb/FakeAVCeleb...
Name: utterance_file, dtype: object

In [ ]:
merged_df = pd.merge(left=wav2lip_df, right=real_df[['source', 'utterance_file']], how='left',
                      left_on=['target1'], right_on=['source'])

In [ ]:
metadata_df = merged_df

In [ ]:
mp4_filepaths = metadata_df.filepath.to_list()
wav_filepaths = [mp4_filepath.replace('mp4','wav') for mp4_filepath in mp4_filepaths]

In [ ]:
metadata_df.columns

Index(['source_x', 'target1', 'target2', 'method', 'category', 'type', 'race',
       'gender', 'filename', 'path', 'filepath', 'source_y', 'utterance_file'],
      dtype='object')

In [ ]:
metadata_df = metadata_df.rename(columns={'source_x': 'source', 
                                          'path': 'directory_path',
                                          'filepath': 'mp4_filepath'})
metadata_df = metadata_df.assign(wav_filepaths=wav_filepaths)
metadata_df = metadata_df.drop(['source_y'], axis=1)

In [ ]:
filename = '/content/drive/MyDrive/FakeAVCeleb/FakeAVCeleb_v1.2/wav2lip_metadata.csv'
metadata_df.to_csv(filename)